<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Dijon

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Dijon_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Dijon - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index

In [7]:
class DijonSpider(scrapy.Spider) :
    # Name of spider
    name = 'Dijon'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=dijon&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=dijon&search_pageview_id=15c43bf6873c02b4&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(DijonSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
dijon = pd.read_json('results_hotels/Dijon_multipage.json')

In [11]:
len(dijon)

125

In [12]:
dijon.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Dijon,https://www.booking.com/hotel/fr/alcove-au-cen...,"L'alcôve, au centre ville de Dijon","8,5",Hébergement géré par un particulier
1,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/b-amp-b-dijon...,B&B Hôtel Dijon Centre,"7,5","Le B & B Hôtel Dijon Centre est situé à Dijon,..."
2,Dijon,https://www.booking.com/hotel/fr/hotelf1-dijon...,hotelF1 Dijon Nord,"6,2","L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ..."
3,Dijon,https://www.booking.com/hotel/fr/grand-apparte...,Grand appartement très calme parking et balcon,"8,7",Hébergement géré par un particulier
4,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/agreable-stud...,Le Millotet - Agréable studio à 2 mn Dijon Gare,"9,1",Hébergement géré par un particulier
5,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/authentique-e...,"L'AUTHENTIQUE, en plein centre ville","8,3",Hébergement géré par un particulier
6,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/le-tillot.fr....,le Tillot,"9,0","Situé à Dijon, à seulement 100 mètres de l'égl..."
7,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/campanile-dij...,Campanile Dijon Centre - Gare,"6,8",Le Campanile Dijon Centre - Gare est situé dan...
8,Dijon,https://www.booking.com/hotel/fr/superbe-t4-pr...,Le Vingt-Six - Centre Ville - Parking,"9,0",Hébergement géré par un particulier
9,Dijon,https://www.booking.com/hotel/fr/hotel-balladi...,Hôtel Restaurant initial by balladins Dijon Nord,"7,6",Situé à 12 minutes en voiture du centre-ville ...


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in dijon['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
dijon['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in dijon['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
dijon['latitude'] = lat
dijon['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in dijon['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
dijon['hotel_description_long'] = result_long_description

In [20]:
dijon.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Dijon,https://www.booking.com/hotel/fr/alcove-au-cen...,"L'alcôve, au centre ville de Dijon","8,5",Hébergement géré par un particulier,"47.33042620,5.03295690",47.33042620,5.03295690,\nVous pouvez bénéficier d'une réduction Geniu...
1,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/b-amp-b-dijon...,B&B Hôtel Dijon Centre,"7,5","Le B & B Hôtel Dijon Centre est situé à Dijon,...","47.32281707,5.03688532",47.32281707,5.03688532,\nLe B & B Hôtel Dijon Centre est situé à Dijo...
2,Dijon,https://www.booking.com/hotel/fr/hotelf1-dijon...,hotelF1 Dijon Nord,"6,2","L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ...","47.35142000,5.06925800",47.35142000,5.06925800,"\nL'hotelf1 Dijon Nord se situe à Dijon, à 1 k..."
3,Dijon,https://www.booking.com/hotel/fr/grand-apparte...,Grand appartement très calme parking et balcon,"8,7",Hébergement géré par un particulier,"47.30925182,5.03027045",47.30925182,5.03027045,\nDoté d'une terrasse et offrant une vue sur l...
4,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/agreable-stud...,Le Millotet - Agréable studio à 2 mn Dijon Gare,"9,1",Hébergement géré par un particulier,"47.32322790,5.03110780",47.32322790,5.03110780,\nVous pouvez bénéficier d'une réduction Geniu...
5,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/authentique-e...,"L'AUTHENTIQUE, en plein centre ville","8,3",Hébergement géré par un particulier,"47.32526350,5.03927180",47.32526350,5.03927180,\nVous pouvez bénéficier d'une réduction Geniu...
6,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/le-tillot.fr....,le Tillot,"9,0","Situé à Dijon, à seulement 100 mètres de l'égl...","47.31997200,5.03459900",47.31997200,5.03459900,\nVous pouvez bénéficier d'une réduction Geniu...
7,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/campanile-dij...,Campanile Dijon Centre - Gare,"6,8",Le Campanile Dijon Centre - Gare est situé dan...,"47.32331785,5.02963811",47.32331785,5.02963811,\nLe Campanile Dijon Centre - Gare est situé d...
8,Dijon,https://www.booking.com/hotel/fr/superbe-t4-pr...,Le Vingt-Six - Centre Ville - Parking,"9,0",Hébergement géré par un particulier,"47.32854560,5.05478120",47.32854560,5.05478120,"\nSitué à Dijon, à 100 mètres du parc des expo..."
9,Dijon,https://www.booking.com/hotel/fr/hotel-balladi...,Hôtel Restaurant initial by balladins Dijon Nord,"7,6",Situé à 12 minutes en voiture du centre-ville ...,"47.35121566,5.06966740",47.35121566,5.06966740,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
dijon['hotel_description_long'] = dijon['hotel_description_long'].replace('\n','', regex=True)

In [22]:
dijon.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Dijon,https://www.booking.com/hotel/fr/alcove-au-cen...,"L'alcôve, au centre ville de Dijon","8,5",Hébergement géré par un particulier,"47.33042620,5.03295690",47.33042620,5.03295690,Vous pouvez bénéficier d'une réduction Genius ...
1,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/b-amp-b-dijon...,B&B Hôtel Dijon Centre,"7,5","Le B & B Hôtel Dijon Centre est situé à Dijon,...","47.32281707,5.03688532",47.32281707,5.03688532,"Le B & B Hôtel Dijon Centre est situé à Dijon,..."
2,Dijon,https://www.booking.com/hotel/fr/hotelf1-dijon...,hotelF1 Dijon Nord,"6,2","L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ...","47.35142000,5.06925800",47.35142000,5.06925800,"L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ..."
3,Dijon,https://www.booking.com/hotel/fr/grand-apparte...,Grand appartement très calme parking et balcon,"8,7",Hébergement géré par un particulier,"47.30925182,5.03027045",47.30925182,5.03027045,Doté d'une terrasse et offrant une vue sur la ...
4,"Centre-ville de Dijon, Dijon",https://www.booking.com/hotel/fr/agreable-stud...,Le Millotet - Agréable studio à 2 mn Dijon Gare,"9,1",Hébergement géré par un particulier,"47.32322790,5.03110780",47.32322790,5.03110780,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
dijon['hotel_description_long'][55]

"Situé à Dijon, à moins de 2 km du parc des expositions Dijon Congrexpo et à 5 km de l'église Saint-Philibert, l'ibis budget Dijon Saint Apollinaire propose un jardin et met gratuitement à votre disposition une connexion Wi-Fi dans l'ensemble de ses locaux ainsi qu'un parking privé. Il propose des chambres familiales et une terrasse.Toutes les chambres comprennent une armoire, une télévision à écran plat, une salle de bains privative, du linge de lit et des serviettes.L'ibis budget Dijon Saint sert un petit-déjeuner continental.Parlant français et anglais, le personnel de la réception se fera un plaisir de vous fournir des informations sur la région.Vous séjournerez à 5 km du Zénith de Dijon et à 4,3 km du jardin botanique de l'Arquebuse. L'aéroport de Dole-Jura, le plus proche, est implanté à 48 km. "

In [24]:
# Clean location column
dijon['location'] = ["Dijon" if value!="Dijon" else value for value in dijon["location"]]

In [25]:
dijon.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Dijon,https://www.booking.com/hotel/fr/alcove-au-cen...,"L'alcôve, au centre ville de Dijon","8,5",Hébergement géré par un particulier,"47.33042620,5.03295690",47.33042620,5.03295690,Vous pouvez bénéficier d'une réduction Genius ...
1,Dijon,https://www.booking.com/hotel/fr/b-amp-b-dijon...,B&B Hôtel Dijon Centre,"7,5","Le B & B Hôtel Dijon Centre est situé à Dijon,...","47.32281707,5.03688532",47.32281707,5.03688532,"Le B & B Hôtel Dijon Centre est situé à Dijon,..."
2,Dijon,https://www.booking.com/hotel/fr/hotelf1-dijon...,hotelF1 Dijon Nord,"6,2","L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ...","47.35142000,5.06925800",47.35142000,5.06925800,"L'hotelf1 Dijon Nord se situe à Dijon, à 1 km ..."
3,Dijon,https://www.booking.com/hotel/fr/grand-apparte...,Grand appartement très calme parking et balcon,"8,7",Hébergement géré par un particulier,"47.30925182,5.03027045",47.30925182,5.03027045,Doté d'une terrasse et offrant une vue sur la ...
4,Dijon,https://www.booking.com/hotel/fr/agreable-stud...,Le Millotet - Agréable studio à 2 mn Dijon Gare,"9,1",Hébergement géré par un particulier,"47.32322790,5.03110780",47.32322790,5.03110780,Vous pouvez bénéficier d'une réduction Genius ...


In [26]:
dijon.to_csv("Dijon.csv")